In [2]:
import data
import sympy
from sympy import Symbol as S, diff
from numpy import matrix as M
import numpy
from sympy import And, Ge, Gt, Le, Lt, Rel, S

In [3]:
from data import n
from data import m
from data import c
from data import c0
from data import b
from data import b0
from data import a
from data import xs
from data import xu

from numpy import float64
from scipy.optimize import linprog
from pprint import pprint

In [4]:
EPS = 0.001

In [5]:
x = M([S('x{}'.format(i)) for i in range(1, n+1)]).T

In [6]:
xs_dict = {x[i, 0]: xs[i, 0] for i in range(n)}

In [7]:
f = c0 * x + 0.5 * x.T * b0.T * b0 * x
f = f[0,0]
g = []
for i in range(m):
    g.append((0.5 * x.T * b[i].T * b[i] * x + c[i] * x + a[i])[0,0])

In [8]:
l = M([S('l{}'.format(i)) for i in range(1, n+1)]).T

In [9]:
lp_c = [diff(f, x[i, 0]).subs(xs_dict) for i in range(n)]

In [10]:
active = [i for i in range(m) if abs(g[i].subs(xs_dict)) < EPS]

In [11]:
lp_a_ub = []
lp_b_ub = []

for i in active:
    lp_a_ub.append([diff(g[i], x[ii, 0]).subs(xs_dict) for ii in range(n)])
    lp_b_ub.append(0)

x_bound = []
for i in range(n):
    x_bound.append((-1 if abs(xs[i,0]) > EPS else 0, 1))

In [12]:
res = linprog(lp_c, A_ub=lp_a_ub, b_ub=lp_b_ub, bounds=x_bound)
l0 = M(res.x).T
l0

matrix([[-1.        ],
        [ 0.        ],
        [ 0.31359649],
        [-1.        ],
        [-1.        ],
        [-1.        ],
        [ 0.        ],
        [ 0.        ]])

In [13]:
a = M([diff(f, x[i, 0]).subs(xs_dict) for i in range(n)], dtype=float) * l0
a = a[0,0]
a

-390.31359649122805

In [14]:
b = M([diff(f, x[i, 0]).subs(xs_dict) for i in range(n)], dtype=float) * (xu - xs)
b = b[0,0]
b

-750.0

In [15]:
alpha = -a/2/b if b>0 else 1
alpha

1

In [16]:
t = 1.0
new_x = xu[:]
while True:
    new_x = xs + t * l0 + alpha * t * (xu - xs)
    new_x_dict = {x[i, 0]: new_x[i, 0] for i in range(n)}
    norm = True
    for i in range(m):
        if f.subs(new_x_dict) <= 0:
            norm = False

    norm = norm and (new_x>=0).all()
    if norm and f.subs(new_x_dict) < f.subs(xs_dict):
        break
    t /= 2
new_x

matrix([[ 0.        ],
        [ 0.        ],
        [ 0.15679825],
        [ 0.5       ],
        [ 1.5       ],
        [ 0.5       ],
        [ 0.        ],
        [ 0.        ]])